# 1. Imports

In [1]:
# imports
import collections
import copy
import json
import matplotlib.pyplot as plt
from statistics import median
import numpy as np
import os
import pandas as pd
import pickle
import scipy.stats
import seaborn as sns
import warnings
import matplotlib
%matplotlib
warnings.filterwarnings("ignore")

 
 
font = {
        # 'family' : 'normal',
        # 'weight' : 'bold',
        'size'   : 18}

matplotlib.rc('font', **font)

"""
all function of this notebook(and some other code.) is based on fallowing amazing github repo:

https://github.com/janvanrijn/openml-pimp

So all credits goes for it.
"""

Using matplotlib backend: Qt5Agg


'\nall function of this notebook(and some other code.) is based on fallowing amazing github repo:\n\nhttps://github.com/janvanrijn/openml-pimp\n\nSo all credits goes for it.\n'

In [2]:
import matplotlib.font_manager as fm
# Rebuild the matplotlib font cache
fm._rebuild()

# 2. Functions

In [3]:
def rank_dict(dictionary, reverse=False):
    '''
    Get a dictionary and return a rank dictionary
    for example dic={'a':10,'b':2,'c':6}
    will return dic={'a':1.0,'b':3.0,'c':2.0}
    
    '''
    dictionary = copy.copy(dictionary)
    
    if reverse:
        
        for key in dictionary.keys():
            dictionary[key] = 1 - dictionary[key]
                      
    sortdict = collections.OrderedDict(sorted(dictionary.items()))
    ranks = scipy.stats.rankdata(list(sortdict.values()))
    result = {}
    
    for idx, (key, value) in enumerate(sortdict.items()):
        result[key] = ranks[idx]
        
    return result

In [4]:
def sum_dict_values(a, b, allow_subsets=False):
    '''
    Get two dictionary sum them together!
    '''
    result = {}
    a_total = sum(a.values())
    b_total = sum(b.values())
    a_min_b = set(a.keys()) - set(b.keys())
    b_min_a = set(b.keys()) - set(a.keys())
    
#     if len(b_min_a) > 0:
#         raise ValueError('dict b got illegal keys: %s' %str(b_min_a))
        
#     if not allow_subsets and len(a_min_b):
#         raise ValueError('keys not the same')
        
    for idx in a.keys():
        if idx in b:
            result[idx] = a[idx] + b[idx]
        else:
            result[idx] = a[idx]
            
#     if sum(result.values()) != a_total + b_total:
#         raise ValueError()
        
    return result

In [5]:
def divide_dict_values(d, denominator):
    ''' 
    divide d/demoniator
    '''
    result = {}
    
    for idx in d.keys():
        result[idx] = d[idx] / denominator
        
    return result

In [6]:
def determine_relevant(data, max_items=None, max_interactions=None):



    sorted_values = []
    keys = []
    interactions_seen = 0


    for key in sorted(data, key=lambda k: median(data[k]), reverse=True):
        if '__' in key:
            interactions_seen += 1
            if interactions_seen > max_interactions:
                continue

        sorted_values.append(data[key])
        keys.append(key)


    if max_items is not None:
        sorted_values = sorted_values[:max_items]
        keys = keys[:max_items]

    return sorted_values, keys

In [7]:
def marginal_plots(sorted_values, keys, fig_title):
    sorted_values=sorted_values[0:9]
    plt.figure(figsize=(12,10))
    violins=plt.violinplot(list(sorted_values), list(range(len(sorted_values))),showmeans=True)
    c=0
    for v in violins['bodies']:
        # pinkish purple #ed0dd9 , pale pink (#ffcfdc) , powder blue (#b1d1fc) , light cyan (#acfffc) , light blue green (#7efbb3)
        #pale purple #b790d4 pistachio (#c0fa8b) , pine (#2b5d34) , dark blue green (#005249)   
        dic_colors={0:'#ed0dd9',1:"#b790d4",2:'#ffcfdc',3:'#b1d1fc',4:'#acfffc',5:'#7efbb3',6:'#c0fa8b',7:'#2b5d34',8:'#005249', 9:'#005249'}

        v.set_facecolor(str(dic_colors[c]))
        # v.set_facecolor('#ed0dd9')
        c+=1
        v.set_edgecolor('black')
        v.set_linewidth(1)
        v.set_alpha(1)

    # sns.violinplot(sorted_values, range(len(sorted_values)))
    plt.plot([-0.5, len(sorted_values) - 0.5], [0, 0], 'k-', linestyle='--', lw=1)
    keys = [format_name(key) for key in keys]
    plt.xticks(list(range(len(sorted_values))), list(keys), rotation=20, ha='right')
    plt.ylabel('Variance Contribution')
    ax=plt.gca()
    ax.set_facecolor('#b7c9e2')
    # plt.xlabel("Hyperparameters")
    plt.grid(color='white', linestyle='-', linewidth=2)
    # plt.title(fig_title)
    # plt.show()
    plt.savefig("../output_plots/"+fig_title+".pdf" ,bbox_inches = 'tight',pad_inches = 0, format='pdf')
    plt.close()

In [8]:
def format_name(name):
    '''
    Format hyperparameter names!
    '''
    mapping_short = {
        'strategy': 'imputation',
        'max_features': 'max_features',
        'min_samples_leaf': 'min_samples_leaf',
        'min_samples_split': 'min_samples_split',
        'criterion': 'criterion',
        'learning_rate': 'learning_rate',
        'max_depth': 'max_depth',
        'n_estimators': 'n_estimators',
        'algorithm': 'algorithm',
        "('max_features', 'min_samples_leaf')":"'max_feat.', 'min_s._leaf'",
        "('criterion', 'max_features', 'min_samples_leaf')":"'criterion', 'max_feat.', 'min_s._leaf'",
        "('max_features', 'min_samples_split')":"'criterion','max_feat.', 'min_s._split'",
        "('bootstrap', 'max_features', 'min_samples_leaf')": "'boots.', 'max_feat.', 'min_s._leaf'",
        "('bootstrap', 'max_features')": "'boots.', 'max_feat.'",
        "('bootstrap', 'min_samples_leaf')":"'boots.', 'min_s._leaf'",
        "('imputation', 'max_features', 'min_samples_leaf')": "'imput.', 'max_feat.', 'min_s._leaf'",
        "('imputation', 'max_features')": "'imput.', 'max_feat.'",
        "('imputation', 'min_samples_leaf')": "'imput.', 'min_s._leaf'",
        "('min_samples_leaf', 'min_samples_split')":"'min_s._leaf', 'min_s._split'",
        "('min_samples_leaf', 'n_estimators')":"'min_s._leaf', 'n_esti.'",
        "('learning_rate', 'max_depth')": "'learning_r.', 'max_depth'",
        "('learning_rate', 'max_features')": "'learning_r.', 'max_feat.'",
        "('max_depth', 'n_estimators')":"'max_depth', 'n_esti.'",
        "('learning_rate', 'n_estimators')": "'learning_r., 'n_esti.'",
        "('learning_rate', 'min_samples_leaf')": "'learning_r.', 'min_s._leaf'",
        "('algorithm', 'max_depth', 'learning_rate')":"'alg.', 'max_depth', 'learning_r.'",
        "('algorithm', 'max_depth')": "'alg.', 'max_depth'",
        "('imputation', 'max_depth', 'learning_rate')":"'imput., 'max_depth', 'learning_r.'",
        "('gamma', 'kernel')":"'gamma', 'kernel'",
        "('imputation', 'kernel')":"'imput.', 'kernel'",
        "('imputation', 'tol')":"'imput.', 'tol'",
        "('C', 'imputation')":"'C', 'imput.'",
        "('coef0', 'gamma')":"'coef0', 'gamma'",
        "('coef0', 'imputation')":"'coef0', 'imput.'",
        "('gamma', 'imputation')":"'gamma', 'imput.'",



        }

    parts = name.split('__')
    for idx, part in enumerate(parts):
        if part in mapping_short:
            parts[idx] = mapping_short[part]
 
                
    return ' / '.join(parts)

In [9]:

#df=df.drop(["imputation","time_taken"],axis=1)
def obtain_marginal_contributions(df):
    '''
    This is the main function that calls Top functions
    '''
    
    all_ranks = dict()
    all_tasks = list()
    total_ranks = None
    num_tasks = 0
    marginal_contribution = collections.defaultdict(list)

    lst_datasets=list(df.dataset.unique())

    for dataset in lst_datasets:


        a=df[df.dataset==dataset]
        a=a.drop("dataset",axis=1)
        param=dict()


        for index, row in a.iterrows():
            marginal_contribution[row["param"]].append(row["importance"])
            param.update( {row["param"] : row["importance"]} )

        ranks = rank_dict(param, reverse=True)
        if total_ranks is None:
            total_ranks = ranks
        else:
            total_ranks = sum_dict_values( ranks,total_ranks, allow_subsets=False)
            num_tasks += 1
    total_ranks = divide_dict_values(total_ranks, num_tasks)
    return total_ranks, marginal_contribution, lst_datasets

## 3. Result: 6 plot over 200 datasets for 6 classifiers(obtain_marginal_contributions)

In [10]:
# {"RF","SVM","ET","DT","AB","GB"}
for cls in {"RF","SVM","ET","DT","AB","GB"}:
    df=pd.read_csv("../PerformanceData/"+cls+"_fANOVA_results.csv")
    total_ranks, marginal_contribution, _ = obtain_marginal_contributions(df)
    sorted_values, keys = determine_relevant(marginal_contribution, max_interactions=3)
    marginal_plots(sorted_values, keys, cls+" classifier")